In [1]:
import pickle
import random
import numpy as np
import pandas as pd
import json
from itertools import groupby, chain, permutations, combinations, combinations_with_replacement

In [2]:
reg_model = open('final_reg_model.sav', 'rb')
reg = pickle.load(reg_model)

In [3]:
print(reg.feature_importances_)

[2.41929347e-01 1.43612345e-01 7.83580597e-04 1.01204220e-04
 5.51466436e-04 1.13638592e-04 3.71696041e-04 1.03751745e-04
 3.71916972e-04 6.77853381e-06 6.03588345e-04 7.50785656e-05
 2.19364171e-04 9.87194975e-06 1.86194526e-04 2.39813158e-07
 2.12815938e-05 1.41306741e-07 2.55752461e-04 0.00000000e+00
 1.44637358e-03 8.91452310e-05 1.18383149e-03 3.00839551e-04
 1.24860120e-03 4.77289095e-04 1.04717745e-03 1.09367934e-04
 1.17926897e-03 2.78946344e-04 8.70615991e-04 1.80979634e-04
 7.37889822e-04 2.31624847e-05 8.28889389e-05 8.55959545e-06
 1.23882953e-03 0.00000000e+00 2.86183956e-03 7.29824514e-05
 2.61600130e-03 5.46324888e-04 3.10927582e-03 5.93263334e-04
 3.16268441e-03 1.02182841e-03 2.96723129e-03 6.33471286e-04
 2.99187256e-03 7.01051380e-04 3.38401341e-03 8.47366506e-04
 9.19210283e-04 3.57706625e-04 3.08419419e-03 7.10812975e-04
 1.05666925e-03 3.05657170e-04 7.06847372e-04 1.78276506e-04
 2.67623485e-04 7.11753619e-04 8.07633118e-04 3.68518700e-04
 6.17972949e-04 1.184299

In [4]:
# Generate all possible item combinations
from itertools import combinations_with_replacement 
def generate_item_combinations(items_list):
    combinations = combinations_with_replacement(items_list,3)
    res = []
    for combination in combinations:
        combination = tuple(x for x in combination)
        # check if tuple is in ascending order
        if(combination[0] <= combination[1] and combination[1] <= combination[2]):
            # check if there are no more than 1 basic items in the tuple 
            # note: 0 is considered as a combined item since 1, 2 or even all 3 item slots can be empty
            if(np.sum(np.floor([(x/10) if x else 1 for x in combination]) == 0) <= 1):
                res.append(combination)
    return res

In [5]:
items_list = [10 * i + j for i in range(10) for j in range(i,10)]
item_combinations_list = generate_item_combinations(items_list)
random.sample(item_combinations_list, 10)

[(3, 38, 68),
 (33, 37, 45),
 (14, 14, 24),
 (9, 16, 78),
 (14, 26, 79),
 (13, 27, 69),
 (27, 47, 99),
 (0, 33, 48),
 (15, 23, 45),
 (23, 36, 44)]

In [9]:
def vectorise_items(items):
    item_index = {x:y for x,y in zip(range(1,10),range(0,18,2))}
    res = {}
    for i,item in enumerate(items):
        item_str = str(item)
        vect = np.zeros([18])
        for i in item_str:
            if i != '0':
                first_index = item_index[int(i)]
                if(not vect[first_index]):
                    vect[first_index] = 1
                else:
                    vect[first_index+1] = 1
        res[item] = vect
    res[0] = np.zeros([18])
    return res
# Vector embeddings for all item combinations
# Convert a list of items to vectors
item_vector_dict = vectorise_items(items_list)
def item_vector_lookup(item_list, d=item_vector_dict):
    res = []
    for items in item_list:
        temp_res = []
        for item in items:
            temp_res = temp_res + d[item].tolist()
        res.append(temp_res)
    return res
item_combinations_vector = item_vector_lookup(item_combinations_list)

In [10]:
item_combinations_vector = np.asarray(item_combinations_vector)

In [11]:

# Get n item recommendations and their predicted rank by running random forest model on all possible item combinations
def get_n_item_recommendations(model, item_combinations_list, item_combinations_vector, input_vector, n=5):
    # Duplicate input vector N times, N = #total number of item combinations
    input_vector_stretched = np.tile(input_vector,(item_combinations_vector.shape[0],1))
    # Add every item combination vector to the stretched input vector
    input_vector_with_item_combo = np.insert(input_vector_stretched,3,item_combinations_vector.transpose(),axis=1)
    # Predict rank
    results = reg.predict(input_vector_with_item_combo)
    # sort indices by predicted rank (descending)
    sorted_items_index = np.argsort(results)
    # Return n item combinations with the highest predicted ranks
    return np.asarray([(item_combinations_list[x], results[x]) for x in sorted_items_index[:n]], dtype = object)

In [12]:
with open('full_mapping.json') as json_data:
    fullmap = json.load(json_data)
item_mapping = fullmap['item_mapping_json']
item_mapping['0'] = 'None'

In [13]:
# Get item names for a tuple of item numbers
def get_item_names(item_combo, item_mapping = item_mapping):
    return tuple(item_mapping.get(str(item)) for item in item_combo)

In [34]:
combined_df = pd.read_pickle("final_data_frame_one_hot")
combined_df = combined_df.drop(columns = ["stage"])
# Get a random row of combined_df and remove all item indices
not_item_index = list(chain(range(3),range(58,combined_df.shape[1])))
#x_single = combined_df.iloc[663,not_item_index]
row = random.randint(0,combined_df.shape[0])
x_single = combined_df.iloc[row,~combined_df.columns.str.match(r'^(item|rank)')]
print(x_single)

def get_row_info(row):
    character = 'None'
    for key in combined_df.iloc[row, combined_df.columns.str.match(r'^TFT3_')].keys():
        if combined_df.iloc[row, combined_df.columns.str.match(r'^TFT3_')][key] == 1:
            character = key
            break
    stage = 'None'
    for key in combined_df.iloc[row, combined_df.columns.str.match(r'^stage')].keys():
        if combined_df.iloc[row, combined_df.columns.str.match(r'^stage')][key] == 1:
            stage = key
            break
    gold_spent = combined_df.iloc[row, combined_df.columns.str.match(r'^gold_spent$')][0]
    level = combined_df.iloc[row, combined_df.columns.str.match(r'^level$')][0]
    info_dict = {'Character': character[5:],
                 'Stage': stage,
                 'Gold_spent': gold_spent,
                 'Level': level}
    return info_dict

row_info = get_row_info(row)
input_vector = x_single.to_numpy()

gold_spent          3
level         3.66667
TFT3_Ahri           0
TFT3_Annie          0
TFT3_Ashe           0
               ...   
stage_4             0
stage_5             0
stage_6             0
stage_7             0
stage_8             0
Name: 1989045, Length: 91, dtype: object


In [35]:
recommended_items = get_n_item_recommendations(reg, item_combinations_list, item_combinations_vector, input_vector,n=50)
for k, v in row_info.items():
    print(f'{k}: {v}')
recommended_items

Character: XinZhao
Stage: stage_2
Gold_spent: 3.0
Level: 3.6666666666666665


array([[(22, 56, 78), 3.3166666666666664],
       [(33, 56, 78), 3.3566666666666665],
       [(15, 56, 78), 3.416666666666666],
       [(35, 56, 78), 3.416666666666666],
       [(56, 56, 78), 3.416666666666666],
       [(5, 56, 78), 3.416666666666666],
       [(45, 56, 78), 3.416666666666666],
       [(25, 56, 78), 3.416666666666666],
       [(3, 56, 78), 3.4266666666666663],
       [(26, 56, 78), 3.4266666666666663],
       [(13, 56, 78), 3.4266666666666663],
       [(29, 56, 78), 3.4266666666666663],
       [(18, 56, 78), 3.4266666666666663],
       [(9, 56, 78), 3.4266666666666663],
       [(46, 56, 78), 3.4266666666666663],
       [(36, 56, 78), 3.4266666666666663],
       [(39, 56, 78), 3.4266666666666663],
       [(7, 56, 78), 3.4266666666666663],
       [(44, 56, 78), 3.4266666666666663],
       [(24, 56, 78), 3.4266666666666663],
       [(47, 56, 78), 3.4266666666666663],
       [(27, 56, 78), 3.4266666666666663],
       [(12, 56, 78), 3.4266666666666663],
       [(6, 56, 78), 

In [36]:
for k, v in row_info.items():
    print(f'{k}: {v}')
[get_item_names(x) for x in recommended_items[:,0]]

Character: XinZhao
Stage: stage_2
Gold_spent: 3.0
Level: 3.6666666666666665


[('RapidFirecannon', 'SwordBreaker', 'ProtectorsChestguard'),
 ('RabadonsDeathcap', 'SwordBreaker', 'ProtectorsChestguard'),
 ('GuardianAngel', 'SwordBreaker', 'ProtectorsChestguard'),
 ('LocketoftheIronSolari', 'SwordBreaker', 'ProtectorsChestguard'),
 ('SwordBreaker', 'SwordBreaker', 'ProtectorsChestguard'),
 ('Chain Vest', 'SwordBreaker', 'ProtectorsChestguard'),
 ('FrozenHeart', 'SwordBreaker', 'ProtectorsChestguard'),
 ('TitansResolve', 'SwordBreaker', 'ProtectorsChestguard'),
 ('Needlessly Large Rod', 'SwordBreaker', 'ProtectorsChestguard'),
 ('RunaansHurricane', 'SwordBreaker', 'ProtectorsChestguard'),
 ('HextechGunblade', 'SwordBreaker', 'ProtectorsChestguard'),
 ('LastWhisper', 'SwordBreaker', 'ProtectorsChestguard'),
 ('BladeoftheRuinedKing', 'SwordBreaker', 'ProtectorsChestguard'),
 ('Sparring Gloves', 'SwordBreaker', 'ProtectorsChestguard'),
 ('ChaliceofFavor', 'SwordBreaker', 'ProtectorsChestguard'),
 ('IonicSpark', 'SwordBreaker', 'ProtectorsChestguard'),
 ('JeweledGauntl